In [1]:
import os
import numpy as np
import pickle
import shutil 
import pImpactR as impact
import matplotlib.pyplot as plt
import time

# Impact2MLI

In [2]:
beam,lattice=impact.readInputFile('IOTA.Chris.in')
for i in range(len(lattice)):
    if 'length' in lattice[i].keys():
        lattice[i].n_sckick = int(np.ceil(lattice[i].length*80))
#     if lattice[i].type=='dipole':
#         print(lattice[i])

reading ImpactZ input file (IOTA.Chris.in)
  : mpi task info .............................done
  : simulation control parameters .............done
  : space charge field solver, mesh info ......done
  : dist-type,restart,subcycle,#of state ......done
  : Multiple Charge State info ................done
  : particle distribution info ................done
  : beam reference orbit info .................done
  : converting impact dist to twiss param......done
  : lattice info ..............................done


In [3]:
g0 = beam.kinetic_energy/beam.mass + 1.0
b0 = np.sqrt((g0+1.0)*(g0-1.0))/g0
print(g0,b0)

196.692801440299 0.999987076031854


In [4]:
lattice = [ item for item in lattice if not item.type == 'write_raw_ptcl' ]
lattice = [ item for item in lattice if not item.type == '-8' ]
lattice[0].turns = 1
for i in range(len(lattice)):
    if 'length' in lattice[i].keys():
        lattice[i].n_sckick = int(np.ceil(lattice[i].length*80))
#     if lattice[i].type=='dipole':
#         print(lattice[i])
        
arc = lattice.copy()
arc.pop(1)


             length: 1.8 [m]
           n_sckick: 144 [1]
              n_map: 100 [1]
         strength_t: 0.4
 transverse_scale_c: 0.01
        pipe_radius: 10.0 [m]
       tune_advance: 0.3
               type: 'nonlinear_insert'

In [5]:
M=impact.getTransferMap(beam,arc)
M.to_csv('M.impact.csv',header=False,index=False)
M

,0,1,2,3,4,5
0,0.999969,0.000070,-6.530033e-09,-1.045721e-06,0.0,1.137732e-14
1,-1.454445,0.999930,3.926343e-09,6.763064e-07,0.0,2.508366e-15
2,0.000000,0.000000,9.999954e-01,3.774304e-07,0.0,0.000000e+00
3,0.000000,0.000000,-1.454438e+00,1.000004e+00,0.0,0.000000e+00
4,0.000069,-0.000099,2.949508e-07,3.352577e-05,1.0,4.648161e-07
5,-0.000000,-0.000000,-0.000000e+00,-0.000000e+00,-0.0,1.000000e+00


In [6]:
NL_nu = lattice[1].tune_advance
NL_L  = lattice[1].length
NL_c  = 0.01
NL_t  = 0.4
alfx = np.tan(np.pi*NL_nu)
betx = NL_L/np.sin(2.0*np.pi*NL_nu)
k = 2*alfx/betx
energy=beam.kinetic_energy
gam0 = energy/beam.mass
bet0 = np.sqrt(1.0-1.0/gam0**2)

In [7]:
elemList, lattice = impact.impact2mli(beam,arc)
for item in elemList:
    if item.elem == 'sbend':
        print(item)

Impact elem type loop is not recognized from MLI. skipping...

  elem: 'sbend'
  name: 'dipole1'
     l: 0.8092963858
 angle: 1.047197551
  hgap: 0.029
  fint: 0.5 [1]

  elem: 'sbend'
  name: 'dipole2'
     l: 0.8092963858
 angle: 1.047197551
  hgap: 0.029
  fint: 0.5 [1]

  elem: 'sbend'
  name: 'dipole3'
     l: 0.4305191429
 angle: 0.5235987756
  hgap: 0.029
  fint: 0.5 [1]

  elem: 'sbend'
  name: 'dipole4'
     l: 0.4305191429
 angle: 0.5235987756
  hgap: 0.029
  fint: 0.5 [1]

  elem: 'sbend'
  name: 'dipole5'
     l: 0.8092963858
 angle: 1.047197551
  hgap: 0.029
  fint: 0.5 [1]

  elem: 'sbend'
  name: 'dipole6'
     l: 0.8092963858
 angle: 1.047197551
  hgap: 0.029
  fint: 0.5 [1]

  elem: 'sbend'
  name: 'dipole7'
     l: 0.4305191429
 angle: 0.5235987756
  hgap: 0.029
  fint: 0.5 [1]

  elem: 'sbend'
  name: 'dipole8'
     l: 0.4305191429
 angle: 0.5235987756
  hgap: 0.029
  fint: 0.5 [1]


In [8]:
ptm=impact.MLI.getElem.ptm()
fin=impact.MLI.getElem.fin

In [9]:
elemList=elemList+[ptm,fin]
labor = impact.MLI.buildLabor([lattice,ptm,fin])

# before optimization

In [10]:
rm mli.out

In [11]:
impact.MLI.writeInputfile(elemList,[lattice],labor)
impact.MLI.run()
M,G = impact.MLI.readTransferMap()
M.to_csv('M.mli.csv',header=False,index=False)
M

,1,2,3,4,5,6
1,0.999969,0.000070,0.000000,0.000000e+00,0.0,-1.143531e-06
2,-1.454445,0.999930,0.000000,0.000000e+00,0.0,-2.521151e-07
3,0.000000,0.000000,0.999995,3.774310e-07,0.0,0.000000e+00
4,0.000000,0.000000,-1.454438,1.000004e+00,0.0,0.000000e+00
5,0.000002,-0.000001,0.000000,0.000000e+00,1.0,-1.296627e+00
6,0.000000,0.000000,0.000000,0.000000e+00,0.0,1.000000e+00


In [ ]:
G.head()

In [ ]:
G2_ref = G.loc[28:76].copy()
G2_ref['GP'] = 0
G2_ref.loc[33,'GP'] = 0.5*k/bet0
G2_ref.loc[67,'GP'] = 0.5*k/bet0

G3_ref = G.loc[84:200].copy()
G3_ref['GP'] = 0
G3_ref.loc[104,'GP'] = k/(2.0*bet0*gam0)**2
G3_ref.loc[184,'GP'] = k/(2.0*bet0*gam0)**2

def obj(G):
    return 1.0e30* np.sum((G.loc[28 :76 ,'GP'].values-G2_ref['GP'].values)**2) \
          +1.0e22* np.sum((G.loc[84 :200,'GP'].values-G3_ref['GP'].values)**2) \
          +1.0e12* np.sum((G.loc[210:450,'GP'].values)**2) \
          +        np.sum((G.loc[462:910,'GP'].values)**2)

In [ ]:
obj(G)

# optimized sextupoles

In [ ]:
with open('result.12sext','rb') as f:
    result = pickle.load(f)
result.x = (result.population[0,:] - 0.5)*4.0

In [ ]:
rm mli.out

In [ ]:
i=0
for item in elemList:
    if item.elem=='thlm':
        item.k2l=result.x[i]
        i=i+1
        
impact.MLI.writeInputfile(elemList,[lattice],labor)
impact.MLI.run()
M,G = impact.MLI.readTransferMap()
M

In [ ]:
G.head()

In [ ]:
obj(G)

# turn off sextupoles (thin multipole)

In [ ]:
rm mli.out

In [ ]:
for item in elemList:
    if item.elem=='thlm':
        item.k2l=0.0
        
impact.MLI.writeInputfile(elemList,[lattice],labor)
impact.MLI.run()
M,G = impact.MLI.readTransferMap()
M

In [ ]:
G.head()

In [ ]:
obj(G)

# remove sbend

In [ ]:
rm mli.out

In [ ]:
elemList,latticeList=impact.MLI.removeElems('sbend',elemList,[lattice])
impact.MLI.writeInputfile(elemList,[lattice],labor)
impact.MLI.run()
M,G = impact.MLI.readTransferMap()
M

In [ ]:
G.head()